<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/Capstone/MultiModal_phi_with_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


**Install dependencies**

In [2]:
!pip install -U --quiet numpy transformers datasets tqdm matplotlib wandb torchmetrics torchinfo pillow pytorch-lightning peft bitsandbytes einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

**Imports**

In [3]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull
!git pull

Cloning into 'ERA_V1'...
remote: Enumerating objects: 2206, done.
remote: Counting objects: 100% (799/799), done.
remote: Compressing objects: 100% (437/437), done.
remote: Total 2206 (delta 407), reused 708 (delta 348), pack-reused 1407
Receiving objects: 100% (2206/2206), 276.88 MiB | 23.84 MiB/s, done.
Resolving deltas: 100% (1099/1099), done.
Updating files: 100% (264/264), done.
Already up to date.
fatal: not a git repository (or any of the parent directories): .git


In [5]:
import torchmetrics
import wandb
import os
import sys
import gc
import torch
import pickle
import json
import torchinfo
import torch.nn as nn
import numpy as np
import pytorch_lightning as pl
import torch.multiprocessing as mp
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from pathlib import Path
from typing import Union, List
from torch.cuda.amp import autocast
from matplotlib import pyplot as plt
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from transformers import AutoProcessor, CLIPVisionModel
from pytorch_lightning.callbacks import Callback
from peft import LoraConfig


In [6]:
sys.path.append("ERA_V1/Capstone")
from llava_instruct_dataset import LlavaFinetuneDataset, LlavaCollator, split_data_to_train_and_val
from model_finetune import LitMultiModalPhiFineTune, SimpleLinearBlock, model_summary

**set parameters**

In [9]:
json_path = './data/llava_instruct_150k.json'
batch_size = 20
device = 'cuda' if torch.cuda.is_available() else 'cpu'
projection_layer_in_channels = 768
projection_layer_out_channels = 2560
#seq_len = 72
#num_image_tokens = 49
#max_ques_length = seq_len - (1+num_image_tokens+1) # 1 for image start, 1 for comment
stage1_projection_checkpoints = 'phi2_projection_checkpoints/ckpt_60001.pt'
projection_layer_finetuning_checkpoint_path = '/content/gdrive/MyDrive/ERA_Capstone/phi2_finetune_checkpoints_run2_low_lr/projection_layer_finetuning/projection_layer_ckpt_finetuning_global_step_4001.pt'
finetuned_phi_checkpoint_path = '/content/gdrive/MyDrive/ERA_Capstone/phi2_finetune_checkpoints_run2_low_lr/phi_model_finetuning/adapter_layer_ckpt_finetuning_global_step_4001'

# Define configurations for QLORA finetuning
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, # Load the model in 4 bits
    bnb_4bit_quant_type="nf4", # 4 bit quant type
    bnb_4bit_use_double_quant=True, # double quant saves more bits
    bnb_4bit_compute_dtype=torch.float16, # use bfloat16
    )

# Define the models and tokenizers
clip_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32")
clip_preprocessor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Define multimodal model
multimodal_phi_model = LitMultiModalPhiFineTune(projection_layer_in_channels,
                                                projection_layer_out_channels,
                                                quantization_config)


multimodal_phi_model.projection_layer.load_state_dict(torch.load(projection_layer_finetuning_checkpoint_path))
multimodal_phi_model.llm_model.from_pretrained(multimodal_phi_model.llm_model, finetuned_phi_checkpoint_path)


config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/9.26k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


RuntimeError: No GPU found. A GPU is needed for quantization.